In [1]:
import numpy as np

In [ ]:
def pseudoInverse(jacobian):
    raise NotImplementedError

def filteredUpdate(target, current, filter):
    """
    Calculates a filtered percental update
    target: T
    current: T
    filter: double
    """
    return (1 - filter) * current + filter * target

def filterStep(update_frequency, filter_percentage):
    """
    Calculates the filter step
    update_frequency: double
    filter_percentage: double
    """
    kappa = -1/ (np.log(1 - min(filter_percentage, 0.999999)))
    return 1/ (kappa * update_frequency + 1.0)

def saturateValue(x, x_min, x_max):
    """
    Saturate a variable x with the limits x_min and x_max
    x: double
    x_min: double
    x_max: double
    """
    return min(max(x, x_min), x_max)

def saturateTorqueRate(tau_d_calculated, tau_d_saturated, delta_tau_max):
    """
    Saturate torque rate to not stress the motors
    tau_d_calculated: vector
    tau_d_saturated: vector 
    delta-tau_max: double 
    """
    for i in range(tau_d_calculated.shape[0]):
        difference = tau_d_calculated[i] - tau_d_saturated[i]
        tau_d_saturated[i] += saturateValue(difference -delta_tau_max, delta_tau_max)

class CartesianImpedanceController:
    def __init__(self):
        pass

    def initDesiredPose(self, position_d_target):
        self._cartesian_stiffness_target = 1 

    def setNumberOfJoints(self, n_joints):
        """
        n_joints: int
        """
        if n_joints < 0:
            raise Exception("Number of joints must be positive")

        self._n_joints = n_joints
        self._q = np.zeros((n_joints,))
        self._dq = np.zeros((n_joints,))
        self._jacobian = np.zeros((6, n_joints))
        self._tau_c = np.zeros((n_joints,))

    def setStiffnessInternal(self, stiffness, auto_damping):
        """
        stiffness: 3x1 vector
        auto_damping: boolean
        """
        if (stiffness < 0).any():
            raise Exception("Stiffness values must be positive")
        np.fill_diagonal(stiffness)
        
        if auto_damping:
            self.applyDamping()

    def setStiffness_big(self, t_x, t_y, t_z, auto_dam):
        """
        Idk what to call this

        t_x: double
        t_y: double
        t_z: double
        """
        stiffness = np.array([t_x, t_y, t_z]) 
        self.setStiffnessInternal(stiffness, auto_dam)

    def setDampingFactors(self, d_x, d_y, d_z):
        """
        d_x: double
        d_y: double
        d_z: double
        """
        damping = np.array([d_x, d_y, d_z])
        self.setDampingFactorsInternal(damping)
        if (damping < 0).any():
            raise Exception("Dampin gfactor must not be negative")
        self._damping_factors = damping
        self.applyDamping()

    def applyDamping(self):
        cartesian_stiffness_target_diag = np.diag(self._cartesian_stiffness_target)
        np.fill_diagonal(self._damping_factors * self.dampingRule(cartesian_stiffness_target_diag))
    
    def setReferencePose(self, position_d_target):
        self.Pposition_d_target = position_d_target

    def setFiltering(self, update_frequency, filter_params_stiffness, filter_params_pose, filter_params_wrench):
        self.setUpdateFrequency(update_frequency) 
        self.setFilterValue(filter_params_stiffness, self._filter_params_stiffness)
        self.setFilterValue(filter_params_pose, self._filter_params_pose)
        self.setFilterValue(filter_params_wrench, self._filter_params_wrench)

    def setMaxTorqueDelta(self, d):
        assert d >= 0.0, "Allowed torque change must be positive"
        self._delta_tau_max = d

    def setMaxTorqueDeltaAndFreq(self, d, freq):
        self.setMaxTorqueDelta(d)
        self.setUpdateFrequency(freq)

    def applyWrench(self, cartesian_wrench_target):
        self._cartesian_wrench_target = cartesian_wrench_target

    def calculateCommandedTorques(self, q, dq, position, jacobian):
        self._q = q
        self._dq = dq
        self._position = position
        self._jacobian = jacobian

    def calculateCommandedTorques(self):
        self.updateFilteredStiffness()
        self.updateFilteredPose()
        self.updateFilteredWrentch()

        # compute error
        self.error = self._position - self._position_d

        # kinematic pseudoinverse
        jacobian_transpose_pinv = pseudoInverse()
        tau_task = self._jacobian.T @ (-self._cartesian_stiffness * self._error - self._cartesian_damping * (self._jacobian * self._dq))
        tau_ext = self._jacobian.T @ self._cartesian_wrench

        tau_d = tau_task + tau_ext
        saturateTorqueRate(tau_d, self._tau_c, self._delta_tau_max)
        return self._tau_c


    def getState(self):
        return {
            'q': self._q,
            'dq': self._dq,
            'position': self._position,
            'position_d': self._position_d,
            'cartesian_stiffness': self._cartesian_stiffness,
            'cartesian_damping': self._cartesian_damping
        }

    def getLastcommands(self):
        return self._tau_c

    def getPoseError(self):
        return self._error

    def getDampingRule(self, stiffness):
        return 2 * np.sqrt(stiffness)

    def setUpdateFrequency(self, freq):
        self._update_frequency = max(freq, 0.0)

    def updateFilteredStiffness(self):
        if self._filter_params_stiffness == 1.0:
            self._cartesian_stiffness = self._cartesian_stiffness_target
            self._cartesian_damping = self._cartesian_damping_target
        else:
            step = filterStep(self._update_frequency, self._filter_params_stiffness)
            self._cartesian_stiffness = filteredUpdate(self._cartesian_stiffness_target, self._cartesian_stiffness, step)
            self._cartesian_damping = filteredUpdate(self._cartesian_damping_target, self._cartesian_damping, step)


    def updateFilteredPose(self):
        if self._filter_params_pose == 1.0:
            self._position_d = self._position_d_target
        else:
            step = filterStep(self._update_frequency, self._filter_params_pose)
            self._position_d = filteredUpdate(self._position_d_target, self._position_d, step)

    def updateFilteredWrentch(self):
        if self._filter_params_wrench == 1.0:
            self._cartesian_wrench = self._cartesian_wrench_target
        else:
            step = filterStep(self._update_frequency, self._filter_params_wrench)
            self._cartesian_wrench = filteredUpdate(self._cartesian_wrench_target, self._cartesian_wrench, step)
    